In [1]:
from shearletNN.shearlets import getcomplexshearlets2D
from shearletNN.shearlet_utils import frequency_shearlet_transform, spatial_shearlet_transform, ShearletTransformLoader
from shearletNN.complex_resnet import complex_resnet18, complex_resnet34, complex_resnet50
from shearletNN.layers import CGELU, CReLU

import torch
import torchvision
from torchvision.transforms import v2
from torchvision import transforms

import gc


patch_size = 96
image_size = 128

rows, cols = image_size, image_size


shearlets, shearletIdxs, RMS, dualFrameWeights = getcomplexshearlets2D(	rows, 
                                                                        cols, 
                                                                        1, 
                                                                        3, 
                                                                        1, 
                                                                        0.5,
                                                                        wavelet_eff_support = image_size,
                                                                        gaussian_eff_support = image_size
                                                                        )

shearlets = torch.tensor(shearlets).permute(2, 0, 1).type(torch.complex64).to(0)

In [2]:
class Unraveling:
    def __init__(self, n):
        self.levels = []
        for i in range(0, n // 2):
            level = []
            for j in range(i, n - i):
                level.append((j, i))
                level.append((i, j))

                level.append((j, n - (i + 1)))
                level.append((n - (i + 1), j))

            level = list(set(level))
            self.levels.append((torch.tensor([x for x, _ in level]), torch.tensor([y for _, y in level])))
    
    def __call__(self, x):
        return [x[..., a, b] for a, b in self.levels]

# this is still technically resolution-independent because that property comes from the shearlet transform crop
# in this case we do not actually need variable number of input tokens because we are always extracting a finite amount of information...
# if the whole premise is that we can utilize even larger images then why does it matter that we are using a variable size input?
class Freakformer(torch.nn.Module):
    def __init__(self, n, chans, embed_dim = 384):
        # first we need to build the tokenizer using the Unraveling transformation and some linear layers
        # this is equivalent to the patching layer
        self.unravel = Unraveling(n)
        self.layers = torch.nn.ModuleList([torch.nn.Linear(len(level), embed_dim) for level in self.unravel.layers])
        pass

    def forward(self, x):
        unraveled = self.unravel(x)
        # (B, C, 4 ... 8(n // 2) + 4)
        tokens = torch.stack([layer(level.flatten(1)) for layer, level in zip(self.layers, unraveled)], -2)
        # (B, n // 2, embed_dim)
        pass

In [9]:
from tqdm import tqdm

def train(model, optimizer, loader, accumulate=1):
    model.train()
    loss = torch.nn.CrossEntropyLoss()

    for i, (X, y) in tqdm(enumerate(loader)):
        out = model(X.to(0))
        optimizer.zero_grad()
        l = loss(out, y.to(0)) / accumulate
        l.backward()
        if i % accumulate == (accumulate - 1):
            optimizer.step()
        

def accuracy(output, target, topk=(1,)):
    output = output.to(torch.device('cpu'))
    target = target.to(torch.device('cpu'))
    maxk = max(topk)
    batch_size = target.shape[0]

    _, idx = output.sort(dim=1, descending=True)
    pred = idx.narrow(1, 0, maxk).t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(dim=0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def epoch_accuracy(loader_s, student):
    student.eval()

    out_epoch_s = [accuracy(student(L.to(0)), y)[0].detach().cpu().item() for L, y in loader_s]

    student.train()

    return sum(out_epoch_s) / len(out_epoch_s)

def test(network, test_loader):
    network.eval().to(0)
    test_loss = 0
    correct = 0
    total = 0
    test_losses=[]
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data.to(0))
            test_loss += torch.nn.CrossEntropyLoss()(output, target.to(0)).item()
            pred = output.data.max(1, keepdim=True)[1].cpu()
            correct += pred.eq(target.data.view_as(pred)).sum()
            total += target.shape[0]
        test_loss /= total
        test_losses.append(test_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, total,
        100. * correct / total))

class IndexSubsetDataset:
    def __init__(self, ds, inds):
        self.ds = ds
        self.inds = inds

    def __iter__(self):
        for i in range(len(self.inds)):
            yield self[i]
    
    def __getitem__(self, i):
        return self.ds[self.inds[i]]
    
    def __len__(self):
        return len(self.inds)
    
    
class FreqModel(torch.nn.Module):
    def __init__(
        self,
        img_size = 224,
        in_chans = 3,
        embed_dim = 384,
    ):
        super().__init__()
        self.img_size = (img_size, img_size)
        self.grid_size = (img_size // 2, 1)
        self.num_patches = img_size // 2

        self.unravel = Unraveling(img_size)
        self.layers = torch.nn.ModuleList([torch.nn.Linear(len(level[0]) * in_chans, embed_dim, dtype=torch.complex64) for level in self.unravel.levels])

    def forward(self, x):

        unraveled = self.unravel(x)
        print([(level.flatten(1).shape, layer.weight.shape) for layer, level in zip(self.layers, unraveled)])
        tokens = torch.cat([layer(level.flatten(1)) for layer, level in zip(self.layers, unraveled)], -2)
        
        return x
    

def linearleaves(module):
    # returns a list of pairs of (parent, submodule_name) pairs for all submodule leaves of the current module
    if isinstance(module, torch.nn.Linear):
        return [(module, None)]

    linear_children = []
    for name, mod in module.named_modules():
        if isinstance(mod, torch.nn.Linear):
            linear_children.append((name, module))
    return linear_children

In [4]:
batch_size_train = 256

In [10]:
from shearletNN.complex_deit import Attention, vit_models, Block, FreqEmbed
from shearletNN.layers import CReLU, ComplexLayerNorm
from functools import partial

def repeat3(x):
    return x.repeat(3, 1, 1)[:3]

train_transform = v2.Compose([
    transforms.RandomResizedCrop((image_size, image_size), scale=(0.5, 1.0)),
    # transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
])

val_transform = v2.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
])

ds_train = torchvision.datasets.Caltech101('./', transform=train_transform, download = True)
ds_train = IndexSubsetDataset(ds_train, sum([list(range(len(ds_train)))[i::5] for i in range(1, 5)], []))

ds_val = torchvision.datasets.Caltech101('./', transform=val_transform, download = True)
ds_val = IndexSubsetDataset(ds_val, list(range(len(ds_val)))[0::5])

train_loader = torch.utils.data.DataLoader(
  ds_train,
  batch_size=batch_size_train, shuffle=True, num_workers=0)

shearlets = shearlets[:3]

def shearlet_transform(img):
    return frequency_shearlet_transform(img, shearlets, patch_size)

train_loader = ShearletTransformLoader(train_loader, shearlet_transform)

val_loader = torch.utils.data.DataLoader(
  ds_val,
  batch_size=batch_size_train, shuffle=False)

val_loader = ShearletTransformLoader(val_loader, shearlet_transform)

for x, y in tqdm(train_loader):
    assert list(x.shape) == [batch_size_train, shearlets.shape[0] * 3, patch_size, patch_size], x.shape
    break
print('building model...')

model = vit_models(
    img_size=patch_size,
    in_chans=shearlets.shape[0] * 3,
    patch_size=16,
    embed_dim=384,
    depth=12,
    num_heads=6,
    mlp_ratio=4,
    qkv_bias=True,
    norm_layer=partial(ComplexLayerNorm, eps=1e-6),
    block_layers=Block,
    Attention_block=Attention,
    act_layer=CGELU,
    Patch_layer=FreqEmbed,
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print('training model...')
for epoch in range(128):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=1)
    gc.collect()
    if epoch % 16 == 15:
        test(model, train_loader)
        test(model, val_loader)

Files already downloaded and verified
Files already downloaded and verified


0it [00:00, ?it/s]


building model...


AttributeError: 'vit_models' object has no attribute 'weight'

In [18]:
print('training model...')
for epoch in range(256):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=1)
    gc.collect()
    if epoch % 16 == 15:
        test(model, train_loader)
        test(model, val_loader)

training model...
epoch 0


28it [00:08,  3.38it/s]


epoch 1


28it [00:08,  3.43it/s]


epoch 2


...

epoch 254


28it [00:08,  3.45it/s]


epoch 255


28it [00:08,  3.39it/s]



Test set: Avg. loss: 0.0008, Accuracy: 6532/6941 (94%)


Test set: Avg. loss: 0.0187, Accuracy: 642/1736 (37%)



In [26]:
from shearletNN.complex_deit import Attention, vit_models, Block
from shearletNN.layers import CReLU, ComplexLayerNorm, ComplexConcat, ComplexFlatten
from functools import partial


def repeat3(x):
    return x.repeat(3, 1, 1)[:3]

train_transform = v2.Compose([
    transforms.RandomResizedCrop((image_size, image_size), scale=(0.5, 1.0)),
    # transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
])

val_transform = v2.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    repeat3,
])

ds_train = torchvision.datasets.Caltech101('./', transform=train_transform, download = True)
ds_train = IndexSubsetDataset(ds_train, sum([list(range(len(ds_train)))[i::5] for i in range(1, 5)], []))

ds_val = torchvision.datasets.Caltech101('./', transform=val_transform, download = True)
ds_val = IndexSubsetDataset(ds_val, list(range(len(ds_val)))[0::5])

train_loader = torch.utils.data.DataLoader(
  ds_train,
  batch_size=batch_size_train, shuffle=True, num_workers=0)

shearlets = shearlets[:3]

def shearlet_transform(img):
    return frequency_shearlet_transform(img, shearlets, patch_size)

train_loader = ShearletTransformLoader(train_loader, shearlet_transform)

val_loader = torch.utils.data.DataLoader(
  ds_val,
  batch_size=batch_size_train, shuffle=False)

val_loader = ShearletTransformLoader(val_loader, shearlet_transform)

for x, y in tqdm(train_loader):
    assert list(x.shape) == [batch_size_train, shearlets.shape[0] * 3, patch_size, patch_size], x.shape
    break
print('building model...')

model = torch.nn.Sequential(torch.nn.Flatten(), 
                            torch.nn.Linear(shearlets.shape[0] * 3 * patch_size * patch_size, 2048, dtype=torch.complex64), 
                            CReLU(), 
                            torch.nn.Linear(2048, 1024, dtype=torch.complex64), 
                            CReLU(), 
                            ComplexConcat(), 
                            torch.nn.Linear(2048, 2048),
                            torch.nn.Linear(2048, 1024),
                            torch.nn.Linear(1024, 101))

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print('training model...')
for epoch in range(256):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=1)
    gc.collect()
    if epoch % 16 == 15:
        test(model, train_loader)
        test(model, val_loader)

Files already downloaded and verified
Files already downloaded and verified


0it [00:00, ?it/s]


building model...
training model...
epoch 0


28it [00:05,  5.38it/s]


epoch 1


...

epoch 254


28it [00:05,  5.35it/s]


epoch 255


28it [00:05,  5.40it/s]



Test set: Avg. loss: 0.0083, Accuracy: 3542/6941 (51%)


Test set: Avg. loss: 0.0122, Accuracy: 586/1736 (34%)



In [27]:
for epoch in range(256):
    print('epoch', epoch)
    train(model.to(0), optimizer, train_loader, accumulate=1)
    gc.collect()
    if epoch % 16 == 15:
        test(model, train_loader)
        test(model, val_loader)

epoch 0


28it [00:05,  5.23it/s]


epoch 1


28it [00:05,  5.36it/s]


epoch 2


...

epoch 254


28it [00:05,  5.35it/s]


epoch 255


28it [00:05,  5.35it/s]



Test set: Avg. loss: 0.0056, Accuracy: 4533/6941 (65%)


Test set: Avg. loss: 0.0125, Accuracy: 689/1736 (40%)



In [ ]:
# amplitude:
amp = torch.abs(x)
print((amp.real.max(), amp.real.min()))

phase = torch.arctan(x.imag / x.real)

phase = torch.nan_to_num(torch.arctan(x.imag / x.real), posinf=torch.math.pi / 2, neginf= -torch.math.pi / 2)

print((phase.real.max(), phase.real.min()))

def to_magnitude_phase(x):
    """
    return magnitude/phase representation 
    """
    return torch.complex(torch.abs(x), torch.nan_to_num(torch.arctan(x.imag / x.real), posinf=torch.math.pi / 2, neginf= -torch.math.pi / 2))

(tensor(5.4802, device='cuda:0'), tensor(0., device='cuda:0'))
(tensor(1.5708, device='cuda:0'), tensor(-1.5708, device='cuda:0'))


In [ ]:
x = torch.tensor([[1.5,.0,.0,.0]])
layerNorm = torch.nn.LayerNorm(4, elementwise_affine = False)
y1 = layerNorm(x)
mean = x.mean(-1, keepdim = True)
var = x.var(-1, keepdim = True, unbiased=False)
y2 = (x-mean)/torch.sqrt(var+layerNorm.eps)

torch.allclose(y1, y2)

True

In [ ]:
x = torch.randn((50,20,100))

layerNorm = torch.nn.LayerNorm(x.shape[-1], elementwise_affine = True)
y1 = layerNorm(x)

mean = torch.mean(x, dim=-1, keepdim=True)
var = torch.square(x - mean).mean(dim=-1, keepdim=True)
y2 = ((x - mean) / torch.sqrt(var + layerNorm.eps)) * layerNorm.weight + layerNorm.bias

torch.allclose(y1, y2, atol=1e-5, rtol=1e-5)

True